In [1]:
import os
import sys

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)

if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
# Import Data
import pandas as pd
tweets_df = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-influence-env\twitter-influence\data\01_raw\top_accounts_details.csv")
tweets_df['date'] = pd.to_datetime(tweets_df['creation_date'], format='%a %b %d %H:%M:%S +0000 %Y')
tweets_df['month_year'] = tweets_df['date'].apply(lambda x: x.strftime('%Y-%m'))
tweets_df['date_day'] = tweets_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
tweets_df.head()

,tweet_id,creation_date,text,user_id,user_creation_date,username,name,follower_count,following_count,number_of_tweets,...,retweet_count,quote_count,reply_count,retweet,video_view_count,in_reply_to_status_id,quoted_status_id,date,month_year,date_day
0,1640814094822383617,Tue Mar 28 20:32:19 +0000 2023,Grateful to all the first responders and volun...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,978,31,308,False,NaN,NaN,NaN,2023-03-28 20:32:19,2023-03,2023-03-28
1,1640485381459025920,Mon Mar 27 22:46:08 +0000 2023,We are failing our children. Guns are now the ...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,23288,2187,16869,False,2833072.0,NaN,NaN,2023-03-27 22:46:08,2023-03,2023-03-27
2,1639039698675937281,Thu Mar 23 23:01:30 +0000 2023,I hope their stories inspire you to help us pr...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,362,27,547,False,NaN,1.639040e+18,NaN,2023-03-23 23:01:30,2023-03,2023-03-23
3,1639039601623851009,Thu Mar 23 23:01:07 +0000 2023,To celebrate the 13th anniversary of the Affor...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,2045,152,859,False,442921.0,NaN,NaN,2023-03-23 23:01:07,2023-03,2023-03-23
4,1638956751910215711,Thu Mar 23 17:31:54 +0000 2023,I know so many of you have a story of how acce...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,641,33,318,False,NaN,1.638957e+18,NaN,2023-03-23 17:31:54,2023-03,2023-03-23


In [3]:
tweets_df['username'].describe()

count            4108
unique            399
top       BBCBreaking
freq              100
Name: username, dtype: object

In [4]:
from scripts.text_processing.sentiment_analysis import SentimentAnalyzer

# sentiment analyser, specify model
analyzer = SentimentAnalyzer('cardiffnlp/twitter-roberta-base-sentiment-latest')

# Get sentiment analysis
tweets_with_sentiment = analyzer.get_sentiment(tweets_df)


C:\Users\johna\anaconda3\envs\twitter-influence-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
influence_metrics = tweets_with_sentiment.groupby('user_id').agg(
    {
        'favorite_count': ['sum', 'mean'],
        'retweet_count': ['sum', 'mean'],
        'quote_count': ['sum', 'mean'],
        'reply_count': ['sum', 'mean'],
        'tweet_id': ['count'],
        'name': [lambda x: x.mode().iloc[0] if not x.mode().empty else None],
    }
)

# Flatten the MultiIndex columns
influence_metrics.columns = ['_'.join(col).strip() for col in influence_metrics.columns.values]

# Reset the index to move the 'username' back to a column
influence_metrics.reset_index(inplace=True)
influence_metrics

,user_id,favorite_count_sum,favorite_count_mean,retweet_count_sum,retweet_count_mean,quote_count_sum,quote_count_mean,reply_count_sum,reply_count_mean,tweet_id_count,name_<lambda>
0,428333,53062,5.527292e+02,15198,158.312500,5241,54.593750,13219,137.697917,96,CNN Breaking News
1,759251,31111,3.142525e+02,7766,78.444444,2247,22.696970,14238,143.818182,99,CNN
2,783214,12895176,1.432797e+05,1633633,18151.477778,916167,10179.633333,801782,8908.688889,90,Twitter
3,807095,36194,4.112955e+02,8676,98.590909,1733,19.693182,4318,49.068182,88,The New York Times
4,813286,4583182,4.928153e+04,467980,5032.043011,51158,550.086022,281364,3025.419355,93,Barack Obama
5,2557521,692864,7.873455e+03,57170,649.659091,13272,150.818182,13805,156.875000,88,ESPN
6,5402612,215655,2.156550e+03,37717,377.170000,27542,275.420000,47118,471.180000,100,BBC Breaking News
7,10228272,222573,2.294567e+03,19936,205.525773,5055,52.113402,73747,760.278351,97,YouTube
8,11348282,464893,5.601120e+03,66276,798.506024,4462,53.759036,12278,147.927711,83,NASA
9,14230524,2678559,3.348199e+04,353313,4416.412500,83226,1040.325000,89683,1121.037500,80,Lady Gaga


In [6]:
print(tweets_df['username'].drop_duplicates())

0           BarackObama
17              bsw5020
23          CivicNation
24         MBK_Alliance
29      ObamaFoundation
             ...       
3991              Drake
4000         Aspiration
4002           caffeine
4007       ajayvirmani1
4015      premierleague
Name: username, Length: 399, dtype: object
